# Well-Field Optimization Demo

This notebook runs the differential evolution optimizer and plots the required outputs.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from geometry import generate_wells
from hydraulics import allocate_flows, pressure_at_wells, well_rates
from optimize import optimize_layout
from breakthrough import compute_tof
from plots import plot_layout, plot_pressure_contours, plot_streamlines, plot_bar_metrics


In [ ]:
x_best, metrics, info = optimize_layout()
r_in, r_out, theta0, e1, e2, e3 = x_best
print('Best variables:', x_best)
print('Metrics:', metrics)

injectors, producers = generate_wells(r_in, r_out, theta0, (e1, e2, e3))
wells = injectors + producers
alloc = allocate_flows()
q_rates = well_rates(wells, alloc)
delta_p = pressure_at_wells(wells, q_rates, alloc.viscosity)
tof = compute_tof(injectors, producers, q_rates, alloc.viscosity)

fig, axes = plt.subplots(2, 2, figsize=(12, 10))
plot_layout(axes[0, 0], injectors, producers)
xx, yy, _ = plot_pressure_contours(axes[0, 1], wells, q_rates, alloc.viscosity, r_out)
plot_streamlines(axes[0, 1], xx, yy, wells, q_rates, alloc.viscosity)

labels_dp = [w.label for w in wells]
plot_bar_metrics(axes[1, 0], labels_dp, list(delta_p), 'Δp (relative)')

labels_tof = list(tof.per_producer.keys())
plot_bar_metrics(axes[1, 1], labels_tof, list(tof.per_producer.values()), 't_bt (s)')

plt.tight_layout()
plt.show()
